In [185]:
import numpy as np
import pandas as pd

np.random.seed(42)
def post_refurb_temp(pre_refurb_temp, Q2_to_Q1_ratio):
    # 10.1016/j.energy.2021.122952
    base = 20
    return base+Q2_to_Q1_ratio**(1/1.3)*(pre_refurb_temp-base)

pre_temps = np.linspace(20,100,100)
q_ratios = np.linspace(0.2,1,5)
post_temps = np.array([post_refurb_temp(pre_temps,red) for red in q_ratios])

post_refurb_temp_df = pd.DataFrame(post_temps.T, index=pre_temps, columns=q_ratios*100)
# post_refurb_temp_df.plot()

In [192]:
import plotly.express as px
from figures import sciencify_plotly_fig

def cop(T_sup, T_source, scnd_ordr_ff=0.5):
    return scnd_ordr_ff * (T_sup + 273.15) / ((T_sup + 273.15) - (T_source + 273.15))


N = 8760
# literature values
avg_savings = np.array([.45,.38,.42,.7,])
print(avg_savings.mean(), avg_savings.std())
expected_savings = np.random.normal(avg_savings.mean(), avg_savings.std(), N)
expected_savings[expected_savings<0] = - expected_savings[expected_savings<0]
expected_savings[expected_savings>1] = 1 - expected_savings[expected_savings>1]

# draw pre_refurb_temperatures
pre_ref_Ts = 50 + np.random.random(N) * 20
post_ref_dem_frac = 1 - expected_savings
print(post_ref_dem_frac.mean())
post_ref_Ts = post_refurb_temp(pre_ref_Ts, post_ref_dem_frac)

fig = sciencify_plotly_fig(px.scatter(x=pre_ref_Ts, y=post_ref_Ts, color=post_ref_dem_frac))
# fig.update_layout(
#     yaxis_title="Post refurbishment supply temperature (°C)",
#     xaxis_title="Pre refurbishment supply temperature (°C)",
#     width=500
# )

0.4875 0.12517487767120045
0.5121795301218628


In [187]:
T_Ontario = pd.read_csv("data/canada/CA_provinces_temperatures.csv", index_col=0, parse_dates=True)["Ontario"]
T_Ontario.mean(), T_Ontario.std()
# T_Ontario.plot(kind="hist", bins=100)

(8.271561643835616, 10.677789709747715)

In [188]:
T_source = T_Ontario.values#np.linspace(-20, 20, N)
post_ref_COPs = cop(post_ref_Ts, T_source)
pre_ref_COPs = cop(pre_ref_Ts, T_source)
cop_improvements = post_ref_COPs / pre_ref_COPs
print(cop_improvements.mean() - 1)

# sciencify_plotly_fig(px.scatter(x=T_source, y=[pre_ref_COPs, post_ref_COPs]))

0.46853529085954904


In [189]:
df = pd.DataFrame(
    {
        "T_source": T_source,
        "pre_ref_Ts": pre_ref_Ts,
        "post_ref_dem_frac": post_ref_dem_frac,
    }
)

df["post_ref_Ts"] = post_refurb_temp(df["pre_ref_Ts"], df["post_ref_dem_frac"])
# ["pre_ref_COPs"]
df["post_ref_COPs"] = cop(df["post_ref_Ts"], df["T_source"])
df["pre_ref_COPs"] = cop(df["pre_ref_Ts"], df["T_source"])
df["cop_change"] = df["post_ref_COPs"] / df["pre_ref_COPs"] - 1
# len(df["post_ref_dem_frac"].round(3).unique())
n_digits = 2
df[f"round{n_digits}_dem_frac"] = df["post_ref_dem_frac"].round(n_digits)
df.loc[df["cop_change"]<0,"cop_change"] = pd.NA
df.loc[df["cop_change"]>3,"cop_change"] = pd.NA
df.dropna()
mean_post_ref_COP_change = df.groupby([f"round{n_digits}_dem_frac"])["cop_change"].mean()

In [190]:
import plotly.graph_objects as go

print((cop_improvements - 1).mean())



fig = sciencify_plotly_fig(
    px.scatter(
        x=post_ref_dem_frac,
        y=(cop_improvements - 1) * 100,
        color=T_source,
        # marginal_y="histogram",
        color_continuous_scale=px.colors.diverging.RdBu_r,
        opacity=0.6
    )
)


fig.update_layout(
    yaxis_title="COP increase (%)",
    xaxis_title=r"Demand fraction Q1/Q0 (-)",
    coloraxis_colorbar_title="T_source (°C)",
    width=500,
    xaxis_range=(.15, 1),
    yaxis_range=(0, 350),
    font_family="cm",
    font_size=18,
    legend=dict(
        x=0.4,
        y=0.98,
        bordercolor="gray",
        borderwidth=2,
    ),
)

fig.add_trace(
    go.Scattergl(
        x=mean_post_ref_COP_change.index,
        y=mean_post_ref_COP_change.values * 100,
        mode="lines",
        marker=dict(color="black"),
        name="mean",
    )
)
from scipy.optimize import curve_fit

def square_func(x,a):
    # return a*x**2+b*x+c
    return a*(x-1)**2+1


p_square,v = curve_fit(square_func, post_ref_dem_frac, (cop_improvements-1)*100)


post_ref_dem_frac.sort()
fig.add_trace(
    go.Scattergl(
        x=post_ref_dem_frac,
        y=square_func(post_ref_dem_frac,*p_square),
        mode="lines",
        line=dict(color="#505050", dash="dash"),
        name="quadratic fit",
        
    )
)
fig.write_image("figures/COP_increase.pdf")
fig

0.4685352908595489


In [191]:
p_square

array([180.60698126])